In [ ]:
import requests
import json
import time
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt

# OpenWeatherMap API Configuration
API_KEY = "2e8a8c238b72f0695521a0507766af38"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
CITY_IDS = {
    "Delhi": 1273294,
    "Mumbai": 1275339,
    "Bangalore": 1277333,
    "Kolkata": 1275004,
    "Hyderabad": 1269843
}

# Database Configuration
DB_NAME = "weather_data.db"

# Alerting Thresholds
TEMP_THRESHOLD = 15  # Celsius

def get_weather_data(city_id):
    """Retrieve weather data from OpenWeatherMap API"""
    params = {
        "id": city_id,
        "appid": API_KEY,
        "units": "metric"  # Use metric units for temperature
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # Check if the response contains the expected data
    if response.status_code != 200 or 'main' not in data:
        print(f"Error retrieving data for city ID {city_id}: {data.get('message', 'Unknown error')}")
        return None
    return data

def convert_temperature(temp_in_kelvin):
    """Convert temperature from Kelvin to Celsius"""
    return temp_in_kelvin - 273.15

def calculate_daily_summary(weather_data):
    """Calculate daily summary from weather data"""
    daily_summary = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "avg_temp": 0,
        "max_temp": float("-inf"),
        "min_temp": float("inf"),
        "dominant_weather": ""
    }

    for data in weather_data:
        daily_summary["avg_temp"] += data["temp"]
        daily_summary["max_temp"] = max(daily_summary["max_temp"], data["temp"])
        daily_summary["min_temp"] = min(daily_summary["min_temp"], data["temp"])
        # Determine dominant weather condition
        if daily_summary["dominant_weather"] == "":
            daily_summary["dominant_weather"] = data["weather"]
        elif data["weather"] != daily_summary["dominant_weather"]:
            daily_summary["dominant_weather"] = "Mixed"

    daily_summary["avg_temp"] /= len(weather_data) if weather_data else 1
    return daily_summary

def store_daily_summary(daily_summary):
    """Store daily summary in SQLite database"""
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS daily_summaries
                (date TEXT, avg_temp REAL, max_temp REAL, min_temp REAL, dominant_weather TEXT)""")
    c.execute("INSERT INTO daily_summaries VALUES (?, ?, ?, ?, ?)",
                (daily_summary["date"], daily_summary["avg_temp"], daily_summary["max_temp"],
                 daily_summary["min_temp"], daily_summary["dominant_weather"]))
    conn.commit()
    conn.close()

def check_alerting_thresholds(weather_data):
    """Check if weather data exceeds alerting thresholds"""
    for data in weather_data:
        if data["temp"] > TEMP_THRESHOLD:
            print(f"Temperature exceeds threshold for {data['city']}: {data['temp']} °C")

def visualize_weather_data():
    """Visualize daily weather summaries"""
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT * FROM daily_summaries")
    rows = c.fetchall()
    dates = [row[0] for row in rows]
    avg_temps = [row[1] for row in rows]
    plt.plot(dates, avg_temps)
    plt.xlabel("Date")
    plt.ylabel("Average Temperature (Celsius)")
    plt.title("Daily Weather Summaries")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def main():
    print("Starting main function")
    weather_data = []
    while True:
        for city, city_id in CITY_IDS.items():
            data = get_weather_data(city_id)
            if data:
                weather_data.append({
                    "city": city,
                    "temp": data["main"]["temp"],  # Use the temperature in Celsius
                    "weather": data["weather"][0]["main"]  # Get the main weather condition
                })

        # Calculate daily summary and store it in the database
        daily_summary = calculate_daily_summary(weather_data)
        store_daily_summary(daily_summary)

        # Check if any temperature exceeds the alerting threshold
        check_alerting_thresholds(weather_data)

        # Clear weather data for the next iteration
        weather_data = []

        # Sleep for a specified duration before the next API call
        time.sleep(60 * 60)  # Sleep for 1 hour

if __name__ == "__main__":
    main()

In [1]:
import requests

def get_weather_data(city, api_key):
  base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
  response = requests.get(base_url)
  weather_data = response.json()

  main_weather_condition = weather_data['weather'][0]['main']
  current_temperature_kelvin = weather_data['main']['temp']
  perceived_temperature_kelvin = weather_data['main']['feels_like']
  update_time = weather_data['dt']

  # Convert Kelvin to Celsius
  current_temperature_celsius = current_temperature_kelvin - 273.15
  perceived_temperature_celsius = perceived_temperature_kelvin - 273.15

  print(f"Weather in {city}:")
  print(f"Main weather condition: {main_weather_condition}")
  print(f"Current temperature: {current_temperature_celsius:.2f}°C")  # Format to 2 decimal places
  print(f"Perceived temperature: {perceived_temperature_celsius:.2f}°C")
  print(f"Update time: {update_time}")

city = input("Enter the city")
api_key ="2e8a8c238b72f0695521a0507766af38"
get_weather_data(city, api_key)

Enter the city London


Weather in London:
Main weather condition: Clouds
Current temperature: 12.03°C
Perceived temperature: 11.66°C
Update time: 1729670005
